<a href="https://colab.research.google.com/github/Sivachandran5710/HCL-training/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community

In [2]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.5 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.6 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.0 MB/s eta 0:00:00


In [19]:
!pip install -q transformers datasets rouge-score bert_score nltk

In [22]:
!pip install evaluate transformers torch bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [31]:
# ---------------------- Install dependencies ----------------------
# !pip install -q gradio torch transformers langchain langchain-community langchain-experimental evaluate faiss-cpu sentence-transformers nltk rouge-score bert-score pdfplumber

# ---------------------- Imports ----------------------
import os
import math
from copy import deepcopy
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
import evaluate

# ---------------------- API Setup ----------------------
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# ---------------------- Model Setup (TOP LEVEL) ----------------------
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# ---------------------- Helper Function ----------------------
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []
    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            overlap = prev[-overlap_characters:]
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)
    return overlapped_chunks

# ---------------------- Gradio Functions ----------------------
def answer_question(file, question):
    loader = PDFPlumberLoader(file.name)
    docs = loader.load()
    text_splitter = SemanticChunker(HuggingFaceEmbeddings())
    semantic_chunks = text_splitter.split_documents(docs)
    documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)
    embedder = HuggingFaceEmbeddings()
    vector = FAISS.from_documents(documents, embedder)
    retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

    llm = ChatOpenAI(
        model="meta-llama/llama-3.3-8b-instruct:free",
        temperature=0.7,
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_key=os.environ["OPENAI_API_KEY"],
        request_timeout=60,
    )

    prompt = """
    You are a helpful assistant.
    Use the following pieces of context to answer the question at the end.
    Answer only using the context and be concise (3–4 sentences).

    Context: {context}
    Question: {question}
    Answer:
    """
    QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
    llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=False)

    document_prompt = PromptTemplate(
        input_variables=["page_content", "source"],
        template="Context:\ncontent:{page_content}\nsource:{source}",
    )

    combine_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
    )

    qa = RetrievalQA(
        combine_documents_chain=combine_documents_chain,
        retriever=retriever,
        return_source_documents=True,
        verbose=False,
    )

    result = qa(question)
    answer = result["result"]
    sources = "\n\n---\n\n".join([doc.page_content[:200] + "..." for doc in result["source_documents"]])
    return answer, sources

def benchmark_questions(file, question):
    loader = PDFPlumberLoader(file.name)
    docs = loader.load()

    text_splitter = SemanticChunker(HuggingFaceEmbeddings())
    semantic_chunks = text_splitter.split_documents(docs)
    documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

    embedder1 = HuggingFaceEmbeddings()
    vector1 = FAISS.from_documents(documents, embedder1)
    retriever1 = vector1.as_retriever(search_type="similarity", search_kwargs={"k": 2})

    embedder2 = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vector2 = FAISS.from_documents(documents, embedder2)
    retriever2 = vector2.as_retriever(search_type="similarity", search_kwargs={"k": 2})

    llm = ChatOpenAI(
        model="meta-llama/llama-3.3-8b-instruct:free",
        temperature=0.7,
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_key=os.environ["OPENAI_API_KEY"],
        request_timeout=60,
    )

    prompt = """
    You are a helpful assistant.
    Use the following pieces of context to answer the question at the end.
    Answer only using the context and be concise (3–4 sentences).

    Context: {context}
    Question: {question}
    Answer:
    """
    QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

    llm_chain1 = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=False)
    llm_chain2 = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=False)

    document_prompt = PromptTemplate(
        input_variables=["page_content", "source"],
        template="Context:\ncontent:{page_content}\nsource:{source}",
    )

    combine_documents_chain1 = StuffDocumentsChain(
        llm_chain=llm_chain1,
        document_variable_name="context",
        document_prompt=document_prompt,
    )
    combine_documents_chain2 = StuffDocumentsChain(
        llm_chain=llm_chain2,
        document_variable_name="context",
        document_prompt=document_prompt,
    )

    qa1 = RetrievalQA(combine_documents_chain=combine_documents_chain1, retriever=retriever1, return_source_documents=True)
    qa2 = RetrievalQA(combine_documents_chain=combine_documents_chain2, retriever=retriever2, return_source_documents=True)

    result1 = qa1(question)
    result2 = qa2(question)
    return result1["result"], result2["result"]


# ---------------------- Gradio Interface ----------------------
with gr.Blocks() as demo:
    gr.Markdown("## RAG QA System with File Upload & Benchmark")

    with gr.Tab("Ask Questions"):
        file_input1 = gr.File(label="Upload PDF")
        question_input = gr.Textbox(label="Enter your question", placeholder="Type your question here...")
        answer_output = gr.Textbox(label="Answer")
        source_output = gr.Textbox(label="Sources")
        ask_btn = gr.Button("Get Answer")
        ask_btn.click(answer_question, inputs=[file_input1, question_input], outputs=[answer_output, source_output])

    with gr.Tab("Benchmark Methods"):
        file_input2 = gr.File(label="Upload PDF")
        benchmark_input = gr.Textbox(label="Enter your question")
        method1_output = gr.Textbox(label="Method 1 Answer")
        method2_output = gr.Textbox(label="Method 2 Answer")
        benchmark_btn = gr.Button("Run Benchmark")
        benchmark_btn.click(benchmark_questions, inputs=[file_input2, benchmark_input], outputs=[method1_output, method2_output])


demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30538426d151b20a36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
!ls

deep_learning  sample_data
